# Exploratory Analysis of Kronos Model

This notebook is for experimenting with the Kronos model. We will:
1. Load the pre-trained Kronos model and tokenizer.
2. Fetch financial data using `yfinance`.
3. Use the model to make predictions on the fetched data.
4. Visualize the results.

In [ ]:
!pip install yfinance

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import yfinance as yf

# To import from parent directory
sys.path.append("../")

from model import Kronos, KronosTokenizer, KronosPredictor

### 1. Load Model and Tokenizer